In [9]:
using JuMP
using Gurobi
using TimerOutputs
to = TimerOutput()
N = 5000
A = rand((1:10),(2,N))

2×5000 Array{Int64,2}:
 6   7  7   2  5  2  2   2   5  8  6  …  7  3  5  1  8  3  9  5  2  8  6  6
 6  10  7  10  6  6  6  10  10  2  5     6  5  4  3  3  6  9  3  8  4  7  2

In [10]:
start = time()
gurobi_env = Gurobi.Env()
setparams!(gurobi_env, Heuristics=0.0, Cuts = 0, OutputFlag = 0)
m = Model(() -> Gurobi.Optimizer(gurobi_env))
@variable(m, y[1:N]>=0)
@objective(m, Min, y[1])

con1 = @constraint(m, A[1,1]* y[1] >= 0)
con2 = @constraint(m, A[2,1]* y[1] >= 0)
# println(m)
for t = 2:N
    set_normalized_coefficient(con1, y[t], A[1,t])
    set_normalized_coefficient(con2, y[t], A[2,t])
    @objective(m, Min, sum(y[l] for l = 1:t))
#     println(m)
    optimize!(m)
end
totalTime = time()-start
println(totalTime)

Academic license - for non-commercial use only
26.429999828338623


In [12]:
start = time()
function MP(t,A)
    m = Model(() -> Gurobi.Optimizer(gurobi_env))
    @variable(m, y[1:t]>=0)
    @objective(m, Min, sum(y[l] for l = 1:t))
    con1 = @constraint(m, sum(A[1,l]* y[l] for l=1:t) >= 0)
    con2 = @constraint(m, sum(A[2,l]* y[l] for l=1:t) >= 0)
    optimize!(m)
end

for t = 1:N
    MP(t, A)
end
totalTime = time()-start
println(totalTime)

60.73099994659424
